In [1]:
filePath = '/Users/yamamotokouhei/subjectsData/data20221122'

In [2]:
#import library
import os
import numpy as np
import scipy.io
import pandas as pd
from scipy import signal
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import pickle

In [3]:
# dataクラス
class O_Data:
    def __init__(self, eeg, index, label):
        self.eeg = eeg
        self.index = index
        self.label = label
        self.stack = []

    def push(self, item):
        self.stack.append(item)

o_data65 = list()
R_data = list()  #  実運動のデータを格納
RI_data = list()  # 実運動＋想起運動のデータを格納
All_data = list()  # 全データを格納
#検証用
testo_data65 = list()
testR_data = list()  #  実運動のデータを格納
testRI_data = list()  # 実運動＋想起運動のデータを格納
testAll_data = list()  # 全データを格納

In [4]:
file_name1 = list()  # すべての.matファイルの名前
for file in os.listdir(filePath):
    base, ext = os.path.splitext(file)
    if ext == '.mat':
        file_name1 = sorted(file_name1)
        file_name1.append(file)
print('Filelist1')
print(file_name1)

Filelist1
[' EEG_grasping_22-Nov-2022_sub1-1.mat', ' EEG_grasping_22-Nov-2022_sub1-2.mat', ' EEG_grasping_22-Nov-2022_sub1-3.mat', ' EEG_grasping_22-Nov-2022_sub2-2.mat', ' EEG_grasping_22-Nov-2022_sub2-3.mat', ' EEG_grasping_22-Nov-2022_sub2-1.mat']


In [5]:
# 変数など
data_directry = ''
file_num1 = len(file_name1)

trial_num = 30
all_trial = (file_num1) * trial_num
all_d = 0
CH = 64
#運動実行を読み込む
for s in range(file_num1):
    Dictionary = scipy.io.loadmat(filePath+'/'+file_name1[s])
    for t in range(trial_num):
        for i, key in enumerate(Dictionary.keys()):
            if i > 2:
                a = Dictionary[key]
                b = a[0, t][0][0]
                eeg = b[0][0:64, :]
                index = b[1]
                samplerate = b[2]

                label = b[5][0, 0]
                #label = 1

                o_data = O_Data(eeg, index, label)
                o_data65.append(o_data)
print("data_loading")

data_loading


In [6]:
N = 8200            # サンプル数
dt = 1/1024          # サンプリング間隔
t = np.arange(0, N*dt, dt) # 時間軸
freq = np.linspace(0, 1.0/dt, N) # 周波数軸
filter3 = signal.firwin(numtaps=51, cutoff=[5, 40], fs=1024, pass_zero=False)

In [7]:
for n in range(all_trial):  # all_trial
    number = list()  # restのindex
    time = list()
    all_d = sum(len(v) for v in o_data65[n].index) - 1
    for i in range(all_d + 1):
        time.append(i)
    eeg = o_data65[n].eeg  # トライアルnのeeg
    index = o_data65[n].index  # トライアルnのindex
    label = o_data65[n].label  # トライアルnのlabel
    for i in range(all_d):
        if index[:, i] == 0:
            number.append(i)
    rest_time = len(number)  # rest時間のindex数
    
    # すべてのチャンネルで行う
    for j in range(CH):
        onech_eeg = eeg[j, :]  # 一つのチャンネルのeeg
        number = np.array(number)
        time = np.array(time)

        filter = signal.lfilter(filter3, 1, onech_eeg[0:8100])
        std = scipy.stats.zscore(filter, axis=None)
        F = np.fft.fft(std)
        Amp = np.abs(F)
        All_data.append(Amp[0:240])

        
        if label == 1:
            #右
            R_data.append(1)
        else:
            #左
            R_data.append(0)

R_data = np.array(R_data)
All_data = np.array(All_data)

print(np.shape(R_data))
print(np.shape(All_data))

(11520,)
(11520, 240)


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(All_data, R_data, test_size=0.25)



model =SVC(kernel='linear')
model.fit(X_train,Y_train)

# SVC(C=1.0, class_weight=None, coef0=0.0,
#   decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
#   max_iter=-1, probability=False, random_state=None, shrinking=True,
#   tol=0.001, verbose=False)

print(model.score(X_train, Y_train))
print(model.score(X_test, Y_test))

0.6804398148148149
0.6350694444444445


In [9]:
params = {
    "C":np.arange(0.1,1,0.05),
    "kernel":["linear", "poly", "rbf", "sigmoid"],
    "gamma":np.arange(0.0001,0.1,0.05)
}
grid = GridSearchCV(model, params,scoring="accuracy", cv=5)

grid.fit(X_train, Y_train)
with open('modke.pickle',mode = 'wb')as f:
    pickle.dump(grid,f,protocol = 2)

print(grid.best_score_)
print(grid.best_params_)

pred = grid.predict(X_test)
print(classification_report(Y_test, pred))